In [1]:
import numpy as np, random, math, operator, pandas as pd, matplotlib.pyplot as plt
import time

In [2]:
random.seed(42)

# Setup

La manière dont le code fonctionne:

Il y a la classe `City` qui est en gros juste les coordonnées (x,y) de sa position, ainsi qu'une méthode pour avoir la distance.

La classe Fitness permet de, étant donné une chemin (une `route`), d'avoir la distance de ce chemin grâce à la méthode `routeDistance`, et la "Fitness" de ce chemin par la méthode `routeFitness` (qui est juste l'inverse de la distance).

Ensuite chaque agent est constitué d'un ensemble de méthodes qui permettent d'implémenter sa fonction de recherche, comme le Recuit Simulé ou bien l'algo génétique. Ces fonctions sont au final rassemblées dans la fonction `_step` qui fait une étape de l'algorithme, et la fonction `search` qui fait l'ensemble de la recherche. 
Les paramètres à donner à ces agents, tant à l'initialisation qu'à la recherche par `search`, sont variables selon l'algo utilisé par l'agent.
En particulier, un Agent est initializé avec l'information de combien d'appels à `_step` un appel à `search` va occasionner, dans `self.generations`.

In [3]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [4]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness
    
    


# Agent superclass: Search Agent

In [5]:
class SearchAgent():
    """
    Implements the `update` method common to all search agents. 
    What this method does is, according to the current agent status, decide
    what to do. 
    
    Status:
        IDLE: waits (firt state, will exit at first update() execution)
        OPTM: optimizing (finding a better solution. If stuck in local optima
              the agent will go to COMP status and wait for another agent 
              to find a better optima)
        COMP: best local solution, comparison with other agents. Will only
              exit if another agent finds a better solution
        
    """
    def __init__(self):
        self.status = "IDLE"
        self.route = None
        self.routeDistance = None
        
    def search(self):
        pass
    
    def __repr__(self):
        return self.__class__.__name__
    
    def update(self, best_other_route=None, best_other_routeDistance=None):
        """
        Does an update of the agent, according to its status
        
        Input:
            Either nothing, or
            the best current route and its distance
        Returns:
            Agent's status, current best route, current best distance
        """
        
        if self.status == "IDLE":
            self.route, self.routeDistance = self.search()
#             print(f'{self} is at distance {self.routeDistance}')
            if best_other_route is not None:
                # if another agent found a better route, take that one and 
                # goes back to optimization
                if best_other_routeDistance < self.routeDistance:
                    self.route = best_other_route
                    self.routeDistance = best_other_routeDistance
            self.status = "OPTM"
            return self.status, self.route, self.routeDistance
            
        elif self.status == "OPTM":
#             print(f'{self} is beginning at distance {self.routeDistance}')
            route, routeDistance = self.search()
#             print(f'{self} best distance found is {routeDistance}')
            # if the route distance didn't change much from last udpate() call
            if (routeDistance > 0.95 * self.routeDistance) and (routeDistance < 1.05 * self.routeDistance):
                    self.status = "COMP"
                    return self.status, self.route, self.routeDistance
            if best_other_route is not None:
                # if another agent found a better route, take that one and 
                # goes back to optimization
                if best_other_routeDistance < self.routeDistance:
                    self.route = best_other_route
                    self.routeDistance = best_other_routeDistance
                    self.status = "OPTM"
                    
                    return self.status, self.route, self.routeDistance
                
            self.route = route
            self.routeDistance = routeDistance
            return self.status, self.route, self.routeDistance
        
        elif self.status == "COMP":
            # if the current best route is provided
            if best_other_route is not None:
                # if another agent found a better route, take that one and 
                # goes back to optimization
                if best_other_routeDistance < self.routeDistance:
                    self.route = best_other_route
                    self.routeDistance = best_other_routeDistance
                    self.status = "OPTM"
                    
                    return self.status, self.route, self.routeDistance
                # else just wait for another agent to improve that solution
                else:
                    return self.status, self.route, self.routeDistance
            # this won't happen after the first iteration of the CoopAgent
            else:
                return self.status, self.route, self.routeDistance

    
        
            
        

# GAAGent

In [6]:
class GAAgent(SearchAgent):
    """
    Hyperparameters: 
        intensification: eliteSize
        diversification: popSize, mutationRate
    """
    
    def __init__(self, popSize, eliteSize, mutationRate, population, generations):
        super().__init__()
        self.popSize = popSize
        self.eliteSize = eliteSize
        self.mutationRate = mutationRate
        self.population = population
        self.generations = generations
    
    
    def createRoute(self, cityList):
        return random.sample(cityList, len(cityList))

    def initialPopulation(self, popSize, cityList):
        return [self.createRoute(cityList) for i in range(popSize)]

    def rankRoutes(self, population):
        fitnessResults = {i:Fitness(population[i]).routeFitness() for i in range(len(population))}
        return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

    def selection(self, popRanked, eliteSize):
        df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
        
        selectionResults = [popRanked[i][0] for i in range(eliteSize)]
        for i in range(0, len(popRanked) - eliteSize):
            pick = 100*random.random()
            for i in range(0, len(popRanked)):
                if pick <= df.iat[i,3]:
                    selectionResults.append(popRanked[i][0])
                    break
                    
        return selectionResults


    def matingPool(self, population, selectionResults):
        matingpool = [population[selectionResults[i]] for i in range(len(selectionResults))]
        return matingpool


    def breed(self, parent1, parent2):
        child = []
        childP1 = []
        childP2 = []

        geneA = int(random.random() * len(parent1))
        geneB = int(random.random() * len(parent1))

        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)

        for i in range(startGene, endGene):
            childP1.append(parent1[i])

        childP2 = [item for item in parent2 if item not in childP1]

        child = childP1 + childP2
        return child

    def breedPopulation(self, matingpool, eliteSize):
        children = []
        length = len(matingpool) - eliteSize
        pool = random.sample(matingpool, len(matingpool))

        for i in range(0,eliteSize):
            children.append(matingpool[i])

        for i in range(0, length):
            child = self.breed(pool[i], pool[len(matingpool)-i-1])
            children.append(child)
        return children


    def mutate(self, individual, mutationRate):
        for swapped in range(len(individual)):
            if(random.random() < mutationRate):
                swapWith = int(random.random() * len(individual))

                city1 = individual[swapped]
                city2 = individual[swapWith]

                individual[swapped] = city2
                individual[swapWith] = city1
        return individual


    def mutatePopulation(self, population, mutationRate):
        mutatedPop = []

        for ind in range(0, len(population)):
            mutatedInd = self.mutate(population[ind], mutationRate)
            mutatedPop.append(mutatedInd)
        return mutatedPop

    def _step(self, currentGen, eliteSize, mutationRate):
        popRanked = self.rankRoutes(currentGen)
        selectionResults = self.selection(popRanked, eliteSize)
        matingpool = self.matingPool(currentGen, selectionResults)
        children = self.breedPopulation(matingpool, eliteSize)
        nextGeneration = self.mutatePopulation(children, mutationRate)
        return nextGeneration

    def search(self):
        
        pop = self.initialPopulation(self.popSize, self.population)
        for i in range(self.generations):
            pop = self._step(pop, self.eliteSize, self.mutationRate)
        bestRouteIndex = self.rankRoutes(pop)[0][0]
        bestRoute = pop[bestRouteIndex]
        
        return bestRoute, Fitness(bestRoute).routeDistance()

## Create City List

In [7]:
def create_cityList(size=25):
    random.seed(42)
    return [City(x=int(random.random() * 1000), y=int(random.random() * 1000)) for i in range(size)]

In [8]:
cityList = create_cityList(100)

In [9]:
cityList

[(639,25),
 (275,223),
 (736,676),
 (892,86),
 (421,29),
 (218,505),
 (26,198),
 (649,544),
 (220,589),
 (809,6),
 (805,698),
 (340,155),
 (957,336),
 (92,96),
 (847,603),
 (807,729),
 (536,973),
 (378,552),
 (829,618),
 (861,577),
 (704,45),
 (227,289),
 (79,232),
 (101,277),
 (635,364),
 (370,209),
 (266,936),
 (648,609),
 (171,729),
 (163,379),
 (989,639),
 (556,684),
 (842,775),
 (229,32),
 (315,267),
 (210,942),
 (876,314),
 (655,395),
 (914,458),
 (264,246),
 (561,262),
 (584,897),
 (399,219),
 (997,509),
 (90,47),
 (109,627),
 (792,422),
 (63,381),
 (996,529),
 (971,860),
 (11,720),
 (681,536),
 (266,640),
 (111,434),
 (453,953),
 (875,263),
 (500,178),
 (912,870),
 (298,638),
 (608,152),
 (762,539),
 (778,530),
 (0,324),
 (19,929),
 (878,831),
 (307,57),
 (878,946),
 (85,485),
 (69,760),
 (765,128),
 (475,549),
 (265,872),
 (423,211),
 (539,729),
 (201,311),
 (995,649),
 (438,517),
 (121,224),
 (338,588),
 (230,220),
 (70,631),
 (228,905),
 (859,70),
 (238,668),
 (214,132),
 (9

## Test GA

In [10]:
def test_GAA(route, print_route=False):

    GAA = GAAgent(popSize=100, eliteSize=20, mutationRate=0.01, population=cityList, generations=200)

    chemin, E = GAA.search()
    
    if print_route: print(route)

    print(E)

In [11]:
test_GAA(cityList, print_route=True)

[(639,25), (275,223), (736,676), (892,86), (421,29), (218,505), (26,198), (649,544), (220,589), (809,6), (805,698), (340,155), (957,336), (92,96), (847,603), (807,729), (536,973), (378,552), (829,618), (861,577), (704,45), (227,289), (79,232), (101,277), (635,364), (370,209), (266,936), (648,609), (171,729), (163,379), (989,639), (556,684), (842,775), (229,32), (315,267), (210,942), (876,314), (655,395), (914,458), (264,246), (561,262), (584,897), (399,219), (997,509), (90,47), (109,627), (792,422), (63,381), (996,529), (971,860), (11,720), (681,536), (266,640), (111,434), (453,953), (875,263), (500,178), (912,870), (298,638), (608,152), (762,539), (778,530), (0,324), (19,929), (878,831), (307,57), (878,946), (85,485), (69,760), (765,128), (475,549), (265,872), (423,211), (539,729), (201,311), (995,649), (438,517), (121,224), (338,588), (230,220), (70,631), (228,905), (859,70), (238,668), (214,132), (935,571), (472,784), (807,190), (96,431), (423,467), (729,673), (984,98), (402,339), (

# RSAGent

In [12]:
class RSAgent(SearchAgent):
    """
    Hyperparameters: 
        t_init, t_min, coeff
    """
    
    def __init__(self, route_init, t_init, coeff, generations):
        super().__init__()
        self.n = len(route_init)
        self.route_init = route_init
        self.temp = t_init
        self.coeff = coeff
        self.generations = generations
    
    def _energy(self, route):
        """
        Compute energy of the route = route distance
        """
    
        return Fitness(route).routeDistance()
    
    def _step(self, route, E):
        """
        Does a step of the RS algorithm
        """
        
    
        sommet1 = random.randint(0, self.n-1)
        sommet2 = random.randint(0, self.n-1)

        route_b = route[:]

        temporaire = route_b[sommet1]
        route_b[sommet1] = route[sommet2]
        route_b[sommet2] = temporaire
        E_b = self._energy(route_b)

        if (E_b < E) or (random.random() < math.exp((E - E_b) / self.temp)):
            route = route_b
            E = E_b

        self.temp *= self.coeff
        
        return route, E
        
    
    
    def search(self):
        """

        """
        
        route = self.route_init
        E = self._energy(route)
        
        for i in range(self.generations):
            
            route, E = self._step(route, E)
            
        return route, E
        

## Test RS

In [13]:
def test_RSA(route, print_route = False):

    RSA = RSAgent(route, t_init = 100, coeff = 0.99, generations=10000)
    
    route, E = RSA.search()
    
    if print_route: print(route)

    print(E)

In [14]:
test_RSA(cityList, print_route=True)

[(704,45), (639,25), (608,152), (550,50), (500,178), (655,395), (829,618), (805,698), (999,836), (971,860), (878,946), (584,897), (210,942), (19,929), (69,760), (249,923), (338,588), (378,552), (218,505), (111,434), (63,381), (0,324), (101,277), (230,220), (264,246), (275,223), (315,267), (402,339), (539,729), (556,684), (648,609), (762,539), (778,530), (649,544), (238,668), (266,640), (298,638), (421,278), (399,219), (423,211), (561,262), (914,458), (935,571), (861,577), (847,603), (878,831), (912,870), (842,775), (807,729), (736,676), (729,673), (792,422), (984,98), (892,86), (859,70), (807,190), (635,364), (423,467), (438,517), (475,549), (472,784), (443,861), (453,953), (536,973), (266,936), (228,905), (265,872), (171,729), (220,589), (190,448), (163,379), (201,311), (227,289), (681,536), (995,649), (989,639), (996,529), (997,509), (214,132), (90,47), (92,96), (26,198), (70,631), (11,720), (109,627), (85,485), (96,431), (79,232), (121,224), (370,209), (876,314), (957,336), (875,263

# TabouAgent

In [15]:
class TAAgent(SearchAgent):
    
    
    def permutation(self,route,i,j): #renvoie le voisin associé à la permutation (i,j) 
        new_route = route.copy()
        (new_route[i],new_route[j])=(new_route[j],new_route[i])
        return(new_route)
    
    def poids (self,route): 
        return Fitness(route).routeFitness() #on cherche à maximiser le fitness qui est supérieur entre 0 (chemin
    #de longueur infinie). C'est notre fonction coût

    def __init__(self, route_init , generations):
        super().__init__()
        self.n = len(route_init)
        self.route_init = route_init
        self.generations = generations
        self.tabou = [(0,0),(1,1)]
        self.meilleur_poids_global = 0 #garde en mémoire la meilleure itération jusqu'à maintenant
        self.meilleure_route_globale=self.route_init.copy()
        
    def _step (self, route):
        
        meilleur_poids_t =0 #ces variables gardent en mémoire le meilleur voisin de la solution considérée à cette étape
        meilleure_route_t= []
        perm=(0,0) #repère la permutation
        
        for i in range (self.n):
            for j in range(i+1,self.n):
                if (i,j) not in self.tabou :
                    new_route = self.permutation(route, i,j)
                    p = self.poids (new_route)
                    if p >= meilleur_poids_t :
                        meilleur_poids_t = p
                        perm = (i,j)
                        meilleure_route_t = new_route
                        
        self.tabou.append(perm) #maj de l'ago tabou
        m = len(self.tabou)
        if m >= self.n *(self.n -1) / 2 :
            del self.tabou[0] #on se contraint à un nombre maximal de déplacements interdits pour que l'algorithme ne 
            #se retrouve pas "coincé" dans toutes les directions
        
        if meilleur_poids_t >= self.meilleur_poids_global : #si le voisin trouvé est meilleur que la solution
            #en mémoire, on met à jour cette dernière
            self.meilleure_route_globale = meilleure_route_t
            self.meilleur_poids_global=meilleur_poids_t
            self.E = Fitness(self.meilleure_route_globale).routeDistance()
        return (meilleure_route_t)
    
    def search (self):
        route = self.route_init
        for k in range (self.generations) :
            route= self._step(route) #on repart du dernier "meilleur voisin" considéré

        return self.meilleure_route_globale, Fitness(self.meilleure_route_globale).routeDistance()
   
                

## Test Tabou

In [16]:
def test_TAA(route, print_route=False):

    TAA = TAAgent(route,generations = 100)

    route, E = TAA.search()
    
    if print_route: print(route)

    print(E)

In [17]:
test_TAA(cityList, print_route=True)

[(561,262), (608,152), (500,178), (550,50), (421,29), (639,25), (704,45), (809,6), (859,70), (892,86), (984,98), (957,336), (996,529), (989,639), (995,649), (999,836), (971,860), (912,870), (807,729), (649,544), (423,467), (227,289), (201,311), (109,627), (171,729), (228,905), (266,936), (249,923), (265,872), (238,668), (220,589), (230,220), (214,132), (307,57), (765,128), (807,190), (861,248), (875,263), (876,314), (655,395), (635,364), (423,211), (229,32), (90,47), (92,96), (340,155), (792,422), (914,458), (997,509), (778,530), (762,539), (681,536), (378,552), (70,631), (85,485), (190,448), (218,505), (266,640), (298,638), (338,588), (729,673), (736,676), (805,698), (842,775), (878,831), (878,946), (584,897), (536,973), (69,760), (11,720), (19,929), (210,942), (453,953), (443,861), (101,277), (121,224), (79,232), (26,198), (0,324), (63,381), (96,431), (111,434), (163,379), (264,246), (275,223), (315,267), (370,209), (402,339), (438,517), (475,549), (648,609), (829,618), (861,577), (9

# CoopAgent

In [26]:
class CoopAgent():
    """
    This agent is the one that's creating and coordinating all the other agent
    work. 
    It first create the agents with various parameters, and then goes through
    cycles of updates to make the agents cooperate according to their status
    """
    
    def __init__(self, agents, nb_cycles):
        self.agents = agents
        self.nb_cycles = nb_cycles
        self.bestRoute = None
        self.bestRouteDistance = None
    
    
    def cycle(self, print_cycles=False):
        counter = 0
        for cycle in range(self.nb_cycles):
            if print_cycles: 
                print(f'Currently at cycle number {cycle}')
            for agent in self.agents:

                start = time.time()
                status, route, distance = agent.update(self.bestRoute, self.bestRouteDistance)
                end = time.time()
#                 print(f'Time taken by agent {agent} is: {end-start}')
                if (not self.bestRouteDistance) or distance < self.bestRouteDistance:
                    if print_cycles:
                        print('-------------------------------')
                        print(f'New best route found by {agent}, with distance {distance}')
                        print('-------------------------------')
                    self.bestRoute = route
                    self.bestRouteDistance = distance
                else:
                    counter += 1
            if counter > len(self.agents)*2:
                print("No more best route found")
                break
                
            
    
    

## Test coop

In [35]:
def test_CoopA(cityList, intensity=1):
    
    TAA1 = TAAgent(cityList, generations=20*intensity)
    RSA1 = RSAgent(cityList, t_init = 100, coeff = 0.99, generations=500* intensity)    
    GAA1 = GAAgent(popSize=100, eliteSize=20, mutationRate=0.01, population=cityList, generations=20*intensity)

    agents = [TAA1, RSA1, GAA1]
    
    CoopA = CoopAgent(agents, 15)
    
    CoopA.cycle(print_cycles=True)

In [36]:
cityList = create_cityList(300)

In [37]:
test_CoopA(cityList, intensity=2)

Currently at cycle number 0


KeyboardInterrupt: 